In [26]:
# Imports
import json
import time
import os

In [27]:
# Configuration of the configuration maker
output_dir = '/home/xams/xams/daqcontrol/ini/'
filename = 'default.ini'

In [28]:
cf = {
    "links": [{
        "type": "V2718",
        "reader": 0,
        "crate": 0,
        "link": 0
    }],

    "boards": [{
        "crate": 0,
        "serial": "077",
        "reader": 0,
        "type": "V1724",
        "vme_address": "Whatever?",
        "link": 0
    }],
    # blt_size {int} : block transfer size in bytes (decimal).
    # V1724 standard: 524288
    # V1730 maximum: 2097152?
    "blt_size": 524288,
    # run_start {int} : 0 - board internal (run start via register), 
    # 1 - s-in (for synchronized boards)
    "run_start": 0,
    "baseline_mode": 0,
    # write_mode {int}: 0-no writing, 1-to file, 2-mongodb
    "write_mode": 2,

    "mongo_collection": "data*",

    "mongo": {
        "write_concern": 0,
        "min_insert_size": 1,
        "database": "xamsdata0",
        "address": "mongodb://localhost:27017"
    },
    "processing_readout_threshold": 0,
    "processing_num_threads": 8,
    "processing_mode": 3,
    "occurrence_integral": 0,

    "compression": 1,
    
     "registers": [
        {
            "comment": "Reset software (all registers)",
            "register": "EF24",
            "board": "-1",
            "value": "1"
        },
        {
            "comment": "BLT event number",
            "register": "EF1C",
            "board": "-1",
            "value": "1"
        },
        {
            "comment": "Enable BERR (error messages)",
            "register": "EF00",
            "board": "-1",
            "value": "10"
        },
        {
            "comment": "Aquisition control register",
            "register": "8100",
            "board": "-1",
            "value": "0" 
            #5: s-in control, 0: board control
        },
        {
            "comment": "Channel enable mask",
            "register": "8120",
            "board": "-1",
            "value": "09" 
            # set to 01 to enable channel 0 only
        },
        {
            "comment": "Measurement time after trigger",
            "register": "8114",
            "board": "-1",
            "value": "1000"
           # 20hex = 32dec = 320 samples = 640 ns
           # 1000 = 81 920 ns

        },
        {
            "comment": "Trigger source enable mask",
            "register": "810C",
            "board": "-1",
            "value": "C1F00003"
            # First digit: C for enable external + software trig
            # Second: coincidence level
            # Third: coincidence window in units of clock cycles of 8 ns. Set to F for 120 ns.
            # Last one (?): channels enable
            # Set to "C0000001" for only trigger 0 and 1
        },
        {
            "comment": "Buffer organisation",
            "register": "800C",
            "board": "-1",
            "value": "6"
            # 800C Buffer organization (Num Buffers = 2^N)
            # A = 5120 samples = 10.24 us
            # 6 = approx. 160 us
        },
        {
            "comment": "DAC calibration",
            "register": "1098",
            "board": "-1",
            "value": "1000"
            # 1098 DAC configuration for each channel (NOTE this is actually register 1n98 for all n) 
            # Allows to define a dc offset to be added to the input signal according to the selected range being set.
            # (1000 = negative unipolar + offset, 8000=bipolar, F000=positive
            # ? No idea what this actually does...
        },
        {
            "comment": "Interrupt status ID",
            "register": "EF14",
            "board": "-1",
            "value": "55AA"
            # Interrupt
        },
        {
            "comment": "Interrupt event number",
            "register": "EF18",
            "board": "-1",
            "value": "0"
            # Interrupt when 1 event stored
        },
        {
            "comment": "Channel 0 gain",
            "register": "8028",
            "board": "-1",
            "value": "0"
            # Channel n gain 0x1n28 (either 0 (2 Vpp) or 1 (0.5 Vpp))
            # ONLY V1730
        },
        {
            "comment": "Channel 0 gain",
            "register": "8328",
            "board": "-1",
            "value": "0"
            # Channel n gain 0x1n28 (either 0 (2 Vpp) or 1 (0.5 Vpp))
            # ONLY V1730
        },
        {
            "comment": "Trigger level channel 0",
            "register": "1080",
            "board": "-1",
            "value": "1FE0"
            # 1FF4 (hex) = 8180 (dec)
            # 1FEA (hex) = 8170 (dec)
            # 1FE0 (hex) = 8160 (dec)
            # 1FD6 (hex) = 8150 (dec)
            # 1F40 (hex) = 8000 (dec)
            # 1E78 (hex) = 7800 (dec)
        },
        {
            "comment": "Trigger level channel 1",
            "register": "1180",
            "board": "-1",
            "value": "0000"
            # 1E78 = 7800
        },
        {
            "comment": "Trigger level channel 3",
            "register": "1380",
            "board": "-1",
            "value": "1FE0"
            # 1E78 = 7800
        },
        {
            "comment": "Trigger polarity",
            "register": "8000",
            "board": "-1",
            "value": "40" # Set to 40 for negative, 00 for positive
        }
    ],
}

In [29]:
config_path = os.path.join(output_dir, filename)
with open(config_path, 'w') as outfile:
    json.dump(cf, outfile, indent = 4)